
# Using SQL Agents with LangChain

This notebook demonstrates how to use LangChain's SQL Agent to query an SQL database. We'll use a mock banking dataset stored in SQLite to answer questions about customers, interest rates, and more.

### Objectives:
- Understand how to connect LangChain's SQL Agent to a database.
- Learn to run natural language queries on the database.
- Extract insights from database tables using an LLM-powered agent.

Let's begin!



## Step 1: Import Libraries

We'll start by importing the necessary libraries for working with LangChain's SQL Agent. These include:
- `langchain.agents` for creating the SQL agent.
- `sqlite3` for database interaction.
- `langchain.chat_models` for utilizing OpenAI's chat models.
- `langchain.sql_database` for handling SQL databases.


In [2]:
from langchain.agents import create_sql_agent
import sqlite3
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase


## Step 2: Set Up the Database

Here, we define the path to our SQLite database and initialize the `SQLDatabase` object, which allows LangChain to interact with the database.


In [18]:

db_path = "mock_bank_data.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")



## Step 3: Initialize the Language Model

We'll use OpenAI's GPT-3.5-turbo model with a temperature of 0.0 to ensure deterministic responses. This model powers the SQL agent.


In [19]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


## Step 4: Create the SQL Agent

Now, we create the SQL agent by combining the LLM and the database. The agent is responsible for translating natural language queries into SQL commands and executing them.


In [20]:
sql_agent = create_sql_agent(
        llm=llm,
        db=db,
        agent_type="openai-tools",
        verbose=True,
    )


## Step 5: Query the Database

Here are some examples of queries you can run using the SQL agent:
1. How many customers are there in the database?
2. What is the average interest rate by product type?
3. What was the first payment default average in monetary values for overdue days > 30 in 2024?

The agent automatically handles the SQL generation and execution for these queries.


In [39]:
n_customers = sql_agent.run("How many customers are there?")
print(n_customers)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


customers, products, transactions
Invoking: `sql_db_schema` with `{'table_names': 'customers'}`



CREATE TABLE customers (
	id INTEGER, 
	name TEXT, 
	age INTEGER, 
	gender TEXT, 
	income REAL, 
	employment_status TEXT, 
	education_level TEXT, 
	marital_status TEXT, 
	dependents INTEGER, 
	residential_status TEXT, 
	city TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from customers table:
id	name	age	gender	income	employment_status	education_level	marital_status	dependents	residential_status	city
1	Calvin Wilson	23	Female	21294.18	Unemployed	High School	Widowed	0	Rented	New Sheilashire
2	Joseph Marks	74	Female	111262.39	Employed	High School	Single	1	Rented	Joetown
3	Brandon Williams	51	Female	119563.97	Employed	Bachelor	Divorced	0	Rented	Dawnfurt
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) AS total_customers FROM customers'}`


[(1000,)]There are 1000 customers in the database.

> Finished

In [41]:
avg_interest = sql_agent.run("What is the average interest rate by product type?")
print(avg_interest)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


customers, products, transactions
Invoking: `sql_db_schema` with `{'table_names': 'products'}`



CREATE TABLE products (
	id INTEGER, 
	name TEXT, 
	type TEXT, 
	interest_rate REAL, 
	credit_limit INTEGER, 
	loan_tenure INTEGER, 
	origination_fee REAL, 
	late_payment_fee REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from products table:
id	name	type	interest_rate	credit_limit	loan_tenure	origination_fee	late_payment_fee
1	Show Credit	Credit Card	9.87	18695	30	302.08	53.06
2	Shake Credit	Personal Loan	5.61	9223	21	358.08	111.67
3	Whatever Credit	Personal Loan	9.51	18164	3	433.03	119.21
*/
Invoking: `sql_db_query` with `{'query': 'SELECT type, AVG(interest_rate) AS average_interest_rate FROM products GROUP BY type'}`


[('Credit Card', 9.857272727272727), ('Mortgage', 12.213), ('Overdraft', 12.137857142857143), ('Personal Loan', 10.08)]The average interest rates by product type are as follows:
- Credit Card

In [42]:
fpd = sql_agent.run("What was the first payment default average in monetary values with overdue days>30 in 2024?") 
print(fpd)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


customers, products, transactions
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	id INTEGER, 
	customer_id INTEGER, 
	product_id INTEGER, 
	transaction_date DATE, 
	due_amount REAL, 
	payment_amount REAL, 
	overdue_days INTEGER, 
	payment_mode TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from transactions table:
id	customer_id	product_id	transaction_date	due_amount	payment_amount	overdue_days	payment_mode
1	575	43	2022-12-03	3577.04	3577.04	0	Online
2	884	5	2015-11-13	2644.88	2644.88	0	Check
3	414	22	2021-09-15	948.25	948.25	0	Online
*/
Invoking: `sql_db_query` with `{'query': "SELECT AVG(payment_amount) AS avg_payment_default FROM transactions WHERE overdue_days > 30 AND transaction_date LIKE '2024%'"}`


[(1568.0099999999998,)]The average payment default amount with overdue days greater than 30 in 2024 was approximately $1568.01.

> Finished chain.
The a


## Conclusion

This notebook demonstrated how to use LangChain's SQL Agent to interact with a database using natural language. Feel free to extend this by adding more complex queries or connecting to your own datasets!
